In [1]:
import math
import requests
from google.colab import files

In [2]:
paid_at_from = '2024-07-01T00:00:00-03:00'
paid_at_to = '2024-07-30T23:59:59-03:00'

In [3]:
headers = {"accept": "application/json"}

In [4]:
logfile = 'iugu.csv'

In [5]:
cabecalho = "Código;Situação;Código da Assinatura;Data do pagamento;CPF/CNPJ;Nome;Total"

In [6]:
#paid, refunded, partially_refunded, chargeback
status = 'paid'

In [7]:
url = "https://api.iugu.com/v1/invoices?limit=1&start=0&paid_at_from=" + str(paid_at_from) + "&paid_at_to=" + str(paid_at_to) + "&status_filter=" +str(status) + "&api_token=DDD91DBD410EBCE75AC818E48BB410B1EDF2401F344D8135B30D2DAF05D5D5C2"

response = requests.get(url, headers=headers)
print (response.text)

{"items":[{"id":"20186821EAE74A99BED487C04AB4469F","due_date":"2024-07-11","currency":"BRL","discount_cents":0,"email":"samuelchinellato@hotmail.com","notification_url":null,"return_url":null,"status":"paid","tax_cents":null,"total_cents":69720,"total_paid_cents":69720,"taxes_paid_cents":1888,"paid_at":"2024-07-11T07:55:47-03:00","paid_cents":69720,"cc_emails":null,"payable_with":"credit_card","overpaid_cents":null,"ignore_due_email":true,"ignore_canceled_email":false,"advance_fee_cents":null,"commission_cents":0,"early_payment_discount":false,"order_id":null,"updated_at":"2024-07-11T07:55:47-03:00","credit_card_brand":"master","credit_card_bin":531681,"credit_card_last_4":"9027","credit_card_captured_at":"2024-07-11T07:55:46-03:00","credit_card_tid":"27834858043Q160O7M3C","external_reference":null,"max_installments_value":null,"payer_name":"Samuel","payer_email":"samuelchinellato@hotmail.com","payer_cpf_cnpj":"10865483612","payer_phone":null,"payer_phone_prefix":null,"payer_address_zi

In [8]:
data = response.json()
totalItems = data['totalItems']
totalItems

574

In [9]:
paginas = math.ceil(totalItems/100) + 1
paginas

7

In [10]:
limit = 0

In [11]:
with open(logfile, 'w') as file:
  file.write(cabecalho+'\n')
  for i in range(0,paginas):
    url = "https://api.iugu.com/v1/invoices?limit=100"+ "&start=" + str(limit) + "&paid_at_from=" + str(paid_at_from) + "&paid_at_to=" + str(paid_at_to) + "&status_filter=" +str(status) +"&api_token=DDD91DBD410EBCE75AC818E48BB410B1EDF2401F344D8135B30D2DAF05D5D5C2"
    result = requests.get(url, headers=headers)
    dt = result.json()
    dados = dt["items"]
    for valor in dados:
      file.write(valor['id']+';'+ valor['status']+';' + str(valor['subscription_id'])+';' + valor['paid_at']+';' + str(valor['total_paid_cents'])+ '\n')
    limit += 100


In [12]:
file = open("iugu.csv","r")
Counter = 0
Content = file.read()
CoList = Content.split("\n")

for i in CoList:
    if i:
        Counter += 1

print(f'This is the number of lines in the file:{Counter}')

This is the number of lines in the file:575


In [13]:
files.download('iugu.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##processa arquivo

In [ ]:
import requests
import json
import csv
import time

In [ ]:
url = 'https://afya-pebmed-refact.gateway.linkapi.solutions/v3/iugu'


In [ ]:
headers = {
  'Content-Type': 'application/json'
}

In [ ]:
logfile = 'processamento.txt'

In [ ]:
payload = json.dumps({
  "event": "invoice.status_changed",
  "data": {
    "id": "idfat",
    "status": "sts",
    "account_id": "0B864B780D28411F91D0172DAB585A2B",
    "async_charged": "",
    "subscription_id": "idsubs",
    "payment_method": "iugu_credit_card",
    "paid_at": "dtpaid",
    "paid_cents": "vlr"
  },
  "retry_count": "0"
})

In [ ]:
search_value = 'ReplyError: ERR invalid expire time in setex'

In [ ]:
i = 0

In [ ]:
with open(logfile, 'w') as file:
  with open('iugu.csv', 'r') as txt_file:
      for line_number, content in enumerate(txt_file):
          if line_number:  # pula cabeçalho
              colunas = content.strip().split(';')
              request = url
              body = payload.replace("idfat",colunas[0])
              body1 = body.replace("idsubs",colunas[2])
              body2 = body1.replace("dtpaid",colunas[3])
              body3 = body2.replace("vlr",colunas[6])
              body4 = body3.replace("sts",colunas[1])
              print(body4)

              response = requests.request("POST", url, headers=headers, data=body4)
              print(f'Resposta - OK: {response.text}')
              if search_value in response.text:
                response = requests.request("POST", url, headers=headers, data=body4)
                print(f'Resposta - Erro - segunda tentativa: {response.text}')
                if search_value in response.text:
                  response = requests.request("POST", url, headers=headers, data=body4)
                  print(f'Resposta - Erro - terceira tentativa: {response.text}')
              file.write(f'{colunas[0]};{body4};{response.text}\n')

              time.sleep(1)
              i += 1
              print(i)


IndexError: list index out of range